<a href="https://colab.research.google.com/github/simply-sankalp/GHI-forecast/blob/main/dataset_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>